# Sampling Rig Contractors 2 Small 3 Big - RF

So Seeing how there are 2 small contractors and 3 large contractors, this sampling method will take the entirety of the 2 small contractors and sample the remaining 3 large contractors.

The the smallest of the 3 large contractors will be how many elements are sampled

In [1]:
from time import time
import numpy as np
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import os
import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

## Establishing Path

In [2]:
path = "../SHARED/"

## Recompletion data

In [4]:
# Reading in the data
data_rcompl = pd.read_excel(path + "RF_imputed_TO_TACC_RCOMPL.xlsx")

In [5]:
# Getting all the contractors
unique_rigs = data_rcompl["CONTRACTOR"].unique()

In [6]:
# Counting the number of entries for each contractor
for i in unique_rigs:
    temp_pd = data_rcompl[data_rcompl["CONTRACTOR"] == i]
    print(i, " has ", len(temp_pd), " elements.")

2  has  805  elements.
0  has  259  elements.
4  has  22  elements.
3  has  403  elements.
1  has  165  elements.


In [7]:
# Now partitioning out how many elements to get from each contractor
rcompl_contractor = [
    2,
    1,
    3,
    0,
    4
]

# Taking all from the smallest 2 contractors
# Then for the upper 3, will take the same amount as the 3rd smallest contractor
rcompl_sample = [
    259,
    165,
    259,
    259,
    22
]

for i in range(5):
    print(rcompl_contractor[i], " ", rcompl_sample[i])

2   259
1   165
3   259
0   259
4   22


In [8]:
# Now go ahead and use pandas sample method to get what we need
new_rcompl = pd.DataFrame(columns = data_rcompl.columns)

for i in range(0, len(rcompl_contractor)):
    print(rcompl_contractor[i], " ", rcompl_sample[i])
    temp_pd = data_rcompl[data_rcompl["CONTRACTOR"] == rcompl_contractor[i]]
    temp_pd = temp_pd.sample(n = rcompl_sample[i])
    new_rcompl = new_rcompl.append(temp_pd, ignore_index=True)

2   259
1   165
3   259
0   259
4   22


In [9]:
new_rcompl

,#WELL_NAME,CURRENT_STATUS,JOB_CATEGORY,JOB_START_DATE,JOB_END_DATE,JOB_STATUS,LIKE_KIND,PROJECT_TYPE,PHASE_START_DATE,PHASE_END_DATE,...,PROPPANT_IN_FORMATION,STG_NUMBER,STG_TOP_DEPTH,STG_BOTTOM_DEPTH,STG_CLEAN_VOLUME_PUMPED,STG_SLURRY_VOLUME_PUMPED,BOTTOM_DEPTH_TVD,Rig_Duration (days),Stg_Duration (hrs),Stage_Length (meters)
0,16,0,0,29,13,1,2,0,42,48,...,447000,20,13491,17648,8805,9283,13014,5,2,4157
1,29,0,0,65,19,0,1,0,1,6,...,328750,22,14305,14446,6799,7155,12725,34,1,141
2,20,0,0,46,11,0,1,0,31,22,...,384000,14,14568,14719,7549,7968,11711,7,1,151
3,7,0,0,24,4,0,2,0,23,25,...,291500,4,12812,18522,5776,6082,12043,9,1,5710
4,44,0,0,7,11,1,2,0,8,15,...,377559,27,13154,17323,7826,8214,12876,4,2,4169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959,2,0,0,9,11,1,1,0,12,20,...,366320,16,13975,14127,6993,7411,11799,6,1,152
960,2,0,0,9,9,1,1,0,12,20,...,367508,10,15005,15156,7305,7724,11754,6,1,151
961,2,0,0,9,9,1,1,0,12,20,...,367900,4,16035,16186,7128,7551,11716,6,1,151
962,2,0,0,9,11,1,1,0,12,20,...,367780,8,15348,15499,7068,7483,11742,6,2,151


In [8]:
# Export to shared
new_rcompl.to_excel(path + "RF_RCOMPL_RIG_SCALED.xlsx")

## Stimulation stage
seeing how this is the same just for stimulation stage, comments will be sparse

In [17]:
data_stmstg = pd.read_excel(path + "STMSTG_KNN_IMPUTED.xlsx")

In [18]:
# Getting all the contractors
unique_rigs = data_stmstg["CONTRACTOR"].unique()

In [19]:
# Counting the elements in each contractor
for i in unique_rigs:
    temp_pd = data_stmstg[data_stmstg["CONTRACTOR"] == i]
    print(i, " has ", len(temp_pd), " elements.")

CONTRACTOR - 2019000008  has  3455  elements.
CONTRACTOR - 2019000027  has  8790  elements.
CONTRACTOR - 2019000030  has  3825  elements.
CONTRACTOR - 2019000023  has  1191  elements.
CONTRACTOR - 2019000051  has  460  elements.


In [20]:
stmstg_contractor = [
    'CONTRACTOR - 2019000008',
    'CONTRACTOR - 2019000023',
    'CONTRACTOR - 2019000027',
    'CONTRACTOR - 2019000030',
    'CONTRACTOR - 2019000051'
]

# Using the same strategy as before
# fully take lower 2
# sample from upper 3
stmstg_sample = [
    3455,
    1191,
    3455,
    3455,
    460
]
for i in range(5):
    print(stmstg_contractor[i], " ", stmstg_sample[i])

CONTRACTOR - 2019000008   3455
CONTRACTOR - 2019000023   1191
CONTRACTOR - 2019000027   3455
CONTRACTOR - 2019000030   3455
CONTRACTOR - 2019000051   460


In [21]:
# use pandas sample function to get our sampled data
new_stmstg = pd.DataFrame(columns = data_stmstg.columns)

for i in range(0, len(stmstg_contractor)):
    print(stmstg_contractor[i], " ", stmstg_sample[i])
    temp_pd = data_stmstg[data_stmstg["CONTRACTOR"] == stmstg_contractor[i]]
    temp_pd = temp_pd.sample(n = stmstg_sample[i])
    new_stmstg = new_stmstg.append(temp_pd, ignore_index=True)

CONTRACTOR - 2019000008   3455
CONTRACTOR - 2019000023   1191
CONTRACTOR - 2019000027   3455
CONTRACTOR - 2019000030   3455
CONTRACTOR - 2019000051   460


In [22]:
new_stmstg

,UWI,UWI_SIDETRACK,FRAC_GRADIENT,BREAKDOWN_PRESSURE,HYDROSTATIC_PRESSURE,TREAT_AVG_PRESSURE,TREAT_MAX_PRESSURE,TREAT_MIN_PRESSURE,PROPPANT_DESIGNED,PROPPANT_IN_FORMATION,...,STIM_TREAT_COMPANY,STIM_TREAT_SUPERVISOR,DIVERSION_COMPANY,DIVERSION_METHOD,DELIVERY_MODE,BOTTOM_HOLE_PRESSURE_METHOD,STIMULATION_RESULT,STIMULATION_RESULT_DETAIL,STIMTREAT_ID,FLUID_NAME
0,2019000491,0,1,8074.0,5128.0,9612.0,10069.0,6985.0,520000,519860,...,STIM_TREAT_COMPANY - 2019000002,CARLOS,DIVERSION_COMPANY - 2019000003,Ball,Casing,Calculated,Success,According to Plan,6B12FC05AEF64B6DBC82FB85F26545C1,25# BORATE
1,2019000457,0,1,8880.0,5568.0,10305.0,10862.0,8010.0,520000,522720,...,STIM_TREAT_COMPANY - 2019000002,GABINO,DIVERSION_COMPANY - 2019000003,FRAC PLUG,Casing,Calculated,Success,Changed Plan,C7553B8BFF6242C2A1D80CB195D99CBF,25# XL
2,2019000856,2,1,9542.0,5595.0,10620.0,10890.0,7278.2,420000,420320,...,STIM_TREAT_COMPANY - 2019000002,CANTU,DIVERSION_COMPANY - 2019000006,Frac Plug,Casing,Calculated,Success,According to Plan,86139FBB4FF54F7C881D471BE1684530,X-link gel
3,2019000825,0,1,8822.0,5784.0,10948.0,11142.0,8190.0,420000,419960,...,STIM_TREAT_COMPANY - 2019000002,DUSTIN,DIVERSION_COMPANY - 2019000003,Frac Plug,Casing,Calculated,Success,According to Plan,526D201D36B242778EABDDB55CF588C9,25# Guar Borate
4,2019000984,0,1,8390.0,5828.0,10469.0,10755.0,9580.0,420000,420450,...,STIM_TREAT_COMPANY - 2019000002,GABINO,DIVERSION_COMPANY - 2019000003,Ball,Casing,Calculated,Success,According to Plan,78BE7B2C2744475989B31AB2A1BD69F4,25# XL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12011,2019000063,0,1,6475.0,5126.0,9687.0,10020.0,7780.0,420000,423470,...,STIM_TREAT_COMPANY - 2019000007,GAMBINO,DIVERSION_COMPANY - 2019000011,Frac Plug,Casing,Calculated,Success,According to Plan,FBE19C3D747047E8BAD545C95DCB8C7A,25# Guar Borate
12012,2019000059,0,1,6715.0,5048.0,10293.0,10409.0,7447.8,420000,419570,...,STIM_TREAT_COMPANY - 2019000007,ERNIE,DIVERSION_COMPANY - 2019000011,Frac Plug,Casing,Calculated,Success,According to Plan,B684A7FF7A18416C9642DE466B7881F7,25# Guar Borate
12013,2019000061,0,1,6806.0,5246.0,10237.0,10402.0,7780.0,420000,420596,...,STIM_TREAT_COMPANY - 2019000007,JOSE,DIVERSION_COMPANY - 2019000011,Frac Plug,Casing,Calculated,Success,According to Plan,BC6EF615A917458EB118A6E01EAC7CF7,X-link gel
12014,2019000063,0,1,5745.0,5101.0,9757.0,9992.0,7780.0,420000,420220,...,STIM_TREAT_COMPANY - 2019000007,GABINO,DIVERSION_COMPANY - 2019000008,Frac Plug,Casing,Calculated,Success,According to Plan,3A530E6ECDE447F8A905AAE848564478,25# Guar Borate


In [23]:
# Export our data
new_stmstg.to_excel(path + "STMSTG_RIG_SCALED.xlsx")